<h1 align='center'><u>Spam Detection HW</u></h1>



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the dataset  

spam dataset from kaggle which can be found from [this](https://www.kaggle.com/uciml/sms-spam-collection-dataset) link. 


In [2]:
import pandas as pd
from pathlib import Path
import zipfile
import tarfile

In [3]:
basepath= '/content/drive/MyDrive/Colab_Notebooks/nlpAssignment'
folder= Path(basepath)
folder

PosixPath('/content/drive/MyDrive/Colab_Notebooks/nlpAssignment')

In [4]:
# The spam csv has already been downloaded and saved in the data folder.

spam = folder / 'spam.csv'
!head {spam}

v1,v2,,,
ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",,,
ham,Ok lar... Joking wif u oni...,,,
spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,,,
ham,U dun say so early hor... U c already then say...,,,
ham,"Nah I don't think he goes to usf, he lives around here though",,,
spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, �1.50 to rcv",,,
ham,Even my brother is not like to speak with me. They treat me like aids patent.,,,
ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,,,
spam,WINNER!! As a valued network customer you have been selected to receivea �900 prize reward! To claim call 09061701461. Claim 

In [5]:
# convert to pandas dataframe
spam_file = pd.read_csv(spam,encoding='ISO-8859-1',
                        names= ['label','message','1','2','3'],
                        usecols=['label','message'], header=0)
spam_df = pd.DataFrame(spam_file)
spam_df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
spam_df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [7]:
747/(4825+747)    # only 13% is spam

0.13406317300789664

Only 13% of the data set is "spam". The rest 87% is "ham". This is an unbalanced dataset.

# Provide the metric for evaluating model 


## Evaluation Metric

Since this is an unbalanced dataset, classification accuracy is not appropriate here. I choose F1 score as the metric for model evaluation.
F1 score is the harmonic mean of precision and recall. It takes both false positive and false negatives into account. Therefore, it performs well on an imbalanced dataset.

F1 score conveys the balance between the precision and the recall. Since not only do we need a good recall score (we want spam to be correctly recorded as spam), we also need a good precision score (we also do not want to mislabel ham as spam), F1 score is appropriate.

# Classification Pipelines

    1. Data Preprocessing + Sparse Embeddings (TF-IDF) + ML Model pipeline
    2. Feature Engineering + ML Model pipeline
    3. Featurization (TF-IDF) + Feature Engineering + ML Model pipeline

features can distinguish a spam from a regular email. like Count : Nouns, ProperNouns, AUX, VERBS, Adjectives, named entities, spelling mistakes 

For Sparse embeddings use **tfidf vectorization**. 

In [8]:
!pip install -U spacy

     |████████████████████████████████| 5.9 MB 5.1 MB/s 
     |████████████████████████████████| 623 kB 1.8 MB/s 
     |████████████████████████████████| 10.1 MB 56.4 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 456 kB 50.2 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [9]:
import spacy
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


In [10]:
spacy_folder = Path('/content/drive/MyDrive/Colab_Notebooks/nlpAssignment/SPACY')
model = spacy_folder /'en_core_web_sm-3.1.0'/'en_core_web_sm'/'en_core_web_sm-3.1.0'
nlp = spacy.load(model, disable=['parser'])

In [11]:
class SpacyPreprocessor(BaseEstimator, TransformerMixin):
    np.random.seed(0)
    def __init__(self, lammetize=True, lower=True, remove_stop=True, 
                 remove_punct=True, remove_email=True, remove_url=True, 
                 remove_num=False, stemming = False,
                 add_user_mention_prefix=True, remove_hashtag_prefix=False):
        self.remove_stop = remove_stop
        self.remove_punct = remove_punct
        self.remove_num = remove_num
        self.remove_url = remove_url
        self.remove_email = remove_email
        self.lammetize = lammetize
        self.lower = lower
        self.stemming = stemming
        self.add_user_mention_prefix = add_user_mention_prefix
        self.remove_hashtag_prefix = remove_hashtag_prefix

 # helpfer functions for basic cleaning 

    def basic_clean(self, text):
        
        '''
        This fuction removes HTML tags from text
        '''
        if (bool(BeautifulSoup(text, "html.parser").find())==True):         
            soup = BeautifulSoup(text, "html.parser")
            text = soup.get_text()
        else:
            pass
        return re.sub(r'[\n\r]',' ', text) 

    # helper function for pre-processing with spacy and Porter Stemmer
    
    def spacy_preprocessor(self,texts):

        final_result = []
        nlp = spacy.load(model, disable=['parser','ner'])
        
        ## Add @ as a prefix so that we can separate the word from its token
        prefixes = list(nlp.Defaults.prefixes)

        if self.add_user_mention_prefix:
            prefixes += ['@']

        ## Remove # as a prefix so that we can keep hashtags and words together
        if self.remove_hashtag_prefix:
            prefixes.remove(r'#')

        prefix_regex = spacy.util.compile_prefix_regex(prefixes)
        nlp.tokenizer.prefix_search = prefix_regex.search

        matcher = Matcher(nlp.vocab)
        if self.remove_stop:
            matcher.add("stop_words", [[{"is_stop" : True}]])
        if self.remove_punct:
            matcher.add("punctuation",[ [{"is_punct": True}]])
        if self.remove_num:
            matcher.add("numbers", [[{"like_num": True}]])
        if self.remove_url:
            matcher.add("urls", [[{"like_url": True}]])
        if self.remove_email:
            matcher.add("emails", [[{"like_email": True}]])
            
        Token.set_extension('is_remove', default=False, force=True)

        cleaned_text = []
        for doc in nlp.pipe(texts,batch_size=64,disable=['parser','ner']):
            matches = matcher(doc)
            for _, start, end in matches:
                for token in doc[start:end]:
                    token._.is_remove =True
                    
            if self.lammetize:              
                text = ' '.join(token.lemma_ for token in doc 
                                if (token._.is_remove==False))
            elif self.stemming:
                text =' '.join(PorterStemmer().stem(token.text) for token in doc 
                                if (token._.is_remove==False))
            else:
                text = ' '.join(token.text for token in doc 
                                if (token._.is_remove==False))
                                   
            if self.lower:
                text=text.lower()
            cleaned_text.append(text)
        return cleaned_text

    def fit(self, X,y=None):
        return self

    def transform(self, X, y=None):
        try:
            if str(type(X)) not in ["<class 'list'>","<class 'numpy.ndarray'>"]:
                raise Exception('Expected list or numpy array got {}'.format(type(X)))
            x_clean = [self.basic_clean(text) for text in X]
            x_clean_final = self.spacy_preprocessor(x_clean)
            return x_clean_final
        except Exception as error:
            print('An exception occured: ' + repr(error))

In [12]:
# install pyspellchecker to use in feature engineering 

!pip install pyspellchecker
from spellchecker import SpellChecker
import re

     |████████████████████████████████| 2.7 MB 4.8 MB/s 


In [57]:
class FeatureEngineering(BaseEstimator, TransformerMixin):
    np.random.seed(0)
    def __init__(self, word_count=False, char_count=False, 
                 char_count_wo_space=False, 
                 avg_word_length=True, digit_count=True,
                 punc_count= True, spell_check= True):
        self.word_count = word_count
        self.char_count = char_count
        self.char_count_wo_space = char_count_wo_space
        self.avg_word_length = avg_word_length
        self.digit_count = digit_count
        self.punc_count = punc_count
        self.spell_check = spell_check
        self.spell = SpellChecker()

    def fit(self, X,y=None):
        return self

    #Useful functions

    def wordCount(self,text):
        return len(text.split())

    def charCount(self,text):
        return len(text)

    def charCountWithoutSpace(self,text):
        count = 0
        for word in text.split():
            count += len(word)
        return count

    def avgWordLength(self,text):
        word_length = 0
        for token in text.split():
            word_length += len(token)
        word_count = len(text.split())
        if word_count == 0:
            return 0
        else:
            return word_length/word_count

    def digitCount(self,text):
        count = 0
        for i in text:
            if i.isdigit():
                count += 1
        return count
    
    def punctuationCount(self,text):
      line = re.findall(r'[!"$%&\'()*+,-./:;=#@?[\]^_`{|}~]*', text)     
      string="".join(line)     
      return len(list(string))
      # puncList = [token for token in self.nlpdocs if(token.is_punct)]
      # return len(puncList)

    def spellcheck(self, text,spell):
      misspelled= spell.unknown(text.split())
      return len(misspelled)

    def transform(self, X,y=None):
        try:
            if str(type(X)) not in ["<class 'list'>","<class 'numpy.ndarray'>"]:
                raise Exception('Expected list or numpy array got {}'.format(type(X)))
            final_result = []
            for index,item in enumerate(X):
                self.nlpdocs = None
                res = []
                if self.word_count:
                    res.append(self.wordCount(item))
                if self.char_count:
                    res.append(self.charCount(item))
                if self.char_count_wo_space:
                    res.append(self.charCountWithoutSpace(item))
                if self.avg_word_length:
                    res.append(self.avgWordLength(item))
                if self.digit_count:
                    res.append(self.digitCount(item))
                if self.punc_count:
                    res.append(self.punctuationCount(item))
                if self.spell_check:
                    res.append(self.spellcheck(item,self.spell))

                final_result.append(res)
            return np.array(final_result)
        except Exception as error:
            print('An exception occured: ' + repr(error))

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
# 40% sample
small_df = pd.DataFrame.sample(spam_df, frac= 0.4,replace=False, random_state=0)
small_df.reset_index(drop=True, inplace= True)
df = small_df

# train test split
X= df['message'].values
Y= df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X,Y,random_state=0)
print(f'X_train: {X_train.shape} y_train: {y_train.shape}')
print(f'X_test: {X_test.shape} y_test: {y_test.shape}')


X_train: (1671,) y_train: (1671,)
X_test: (558,) y_test: (558,)


In [17]:
df.head()

,label,message
0,ham,Aight should I just plan to come up later toni...
1,ham,Was the farm open?
2,ham,I sent my scores to sophas and i had to do sec...
3,ham,Was gr8 to see that message. So when r u leavi...
4,ham,In that case I guess I'll see you at campus lodge


In [18]:
import re
from bs4 import BeautifulSoup

from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
from spacy.matcher import Matcher
from spacy.tokens import Token
import nltk
from nltk.stem.porter import PorterStemmer

Data preprocessing + Sparse Embeddings + ML Model pipeline :

I am using XGBoost Classifier for imbalanced classification. 
Since the dataset is unbalanced, we will have to apply scale weights for the positive label "spam". 

In [104]:
# Preprocessing as part of pipeline

classifier_1 = Pipeline([('preprocessor', SpacyPreprocessor(remove_stop=False)),
                  ('vectorizer', TfidfVectorizer(analyzer='word', 
                                                 token_pattern=r"[\S]+")),
                 
                  ('classifier', XGBClassifier(max_depth=4)),
                 ])

param_grid_classifier_1 = {'preprocessor__lammetize' : [True, False],
                'vectorizer__max_features': [500, 1000, 2000],
                'vectorizer__max_df': [0.8, 0.7],
                'classifier__scale_pos_weight': [7, 13]}

In [105]:
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

f1_scorer = make_scorer(f1_score, pos_label="spam")
grid_classifier_1 = GridSearchCV(estimator=classifier_1, 
                                 param_grid=param_grid_classifier_1, 
                                 cv = 5, scoring= f1_scorer)

In [106]:
grid_classifier_1.fit(X_train, y_train)


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        SpacyPreprocessor(add_user_mention_prefix=True,
                                                          lammetize=True,
                                                          lower=True,
                                                          remove_email=True,
                                                          remove_hashtag_prefix=False,
                                                          remove_num=False,
                                                          remove_punct=True,
                                                          remove_stop=False,
                                                          remove_url=True,
                                                          stemming=False)),
                                       ('vectorizer',
                                

In [107]:
print("Best parameters: {}".format(grid_classifier_1.best_params_))
print("\nBest estimator: ", grid_classifier_1.best_estimator_)

print('Train_score: {}'.format(grid_classifier_1.score(X_train,y_train)))
print('Test_score: {}'.format(grid_classifier_1.score(X_test,y_test)))

Best parameters: {'classifier__scale_pos_weight': 7, 'preprocessor__lammetize': True, 'vectorizer__max_df': 0.8, 'vectorizer__max_features': 500}

Best estimator:  Pipeline(memory=None,
         steps=[('preprocessor',
                 SpacyPreprocessor(add_user_mention_prefix=True, lammetize=True,
                                   lower=True, remove_email=True,
                                   remove_hashtag_prefix=False,
                                   remove_num=False, remove_punct=True,
                                   remove_stop=False, remove_url=True,
                                   stemming=False)),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.f...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                            

The Data preprocessing + Sparse Embeddings + ML Model pipeline is giving a decent f1 test score of 0.85. However, the train score is at 0.98 which indicates that there is an overfitting issue.

Feature Engineering + ML Model pipeline :

In [113]:
# Feature Engineering + ML pipeline

classifier_2 = Pipeline([('fe', FeatureEngineering()), 
                         ('classifier', XGBClassifier())])

param_grid_classifier_2 = {
                'fe__word_count':[True,False],
                'fe__digit_count': [True, False],
                'classifier__scale_pos_weight': [7, 8, 13]}

In [114]:
grid_classifier_2 = GridSearchCV(estimator = classifier_2, 
                                 param_grid = param_grid_classifier_2,
                                 cv = 5, scoring= f1_scorer)
grid_classifier_2.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('fe',
                                        FeatureEngineering(avg_word_length=True,
                                                           char_count=False,
                                                           char_count_wo_space=False,
                                                           digit_count=True,
                                                           punc_count=True,
                                                           spell_check=True,
                                                           word_count=False)),
                                       ('classifier',
                                        XGBClassifier(base_score=0.5,
                                                      booster='gbtree',
                                                      colsample_bylevel=1,
                                                 

In [115]:
print("Best parameters: {}".format(grid_classifier_2.best_params_))
print("\nBest estimator: ", grid_classifier_2.best_estimator_)

print('Train_score: {}'.format(grid_classifier_2.score(X_train,y_train)))
print('Test_score: {}'.format(grid_classifier_2.score(X_test,y_test)))

Best parameters: {'classifier__scale_pos_weight': 7, 'fe__digit_count': True, 'fe__word_count': False}

Best estimator:  Pipeline(memory=None,
         steps=[('fe',
                 FeatureEngineering(avg_word_length=True, char_count=False,
                                    char_count_wo_space=False, digit_count=True,
                                    punc_count=True, spell_check=True,
                                    word_count=False)),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='binary:logistic', random_state=0,
                           

The Feature Engineering + ML model Pipeline using grid search is giving  an F1 train score of 0.93, and F1 test score of 0.88. Now this result is definitely an improvement over the previous, classifier_1. There is no problem of overfitting here. 

Data preprocesser + Sparse Embeddings + Feature Engineering + ML Model


In [109]:
# Data preprocesser + Sparse Embeddings + Feature Engineering + ML Model

vectorization= Pipeline([('preprocessor', SpacyPreprocessor(remove_punct=False)),
                         ('vectorizer', TfidfVectorizer(analyzer='word',
                                                      token_pattern=r"[\S]+"))])
feature_engineering = FeatureEngineering()
combined_features = FeatureUnion([('vec', vectorization),
                                  ('fe', feature_engineering)]) 

In [110]:
classifier_3 = Pipeline([('cf', combined_features),
                         ('classifier', XGBClassifier())])

param_grid_classifier_3 = {'cf__vec__preprocessor__lammetize' : [True, False],
                'cf__vec__vectorizer__max_features': [500, 1000, 2000],
                'cf__vec__vectorizer__max_df': [0.8, 0.7],
                'cf__fe__word_count' : [True],
                'cf__fe__digit_count': [True],
                'classifier__scale_pos_weight': [7, 13]}
grid_classifier_3= GridSearchCV(estimator= classifier_3,
                                param_grid= param_grid_classifier_3,
                                cv = 5, scoring= f1_scorer)

In [111]:
grid_classifier_3.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('cf',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('vec',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('preprocessor',
                                                                                         SpacyPreprocessor(add_user_mention_prefix=True,
                                                                                                           lammetize=True,
                                                                                                           lower=True,
                                                                                                           remove_email=True,
                        

In [112]:
print("Best parameters: {}".format(grid_classifier_3.best_params_))

print('Train_score: {}'.format(grid_classifier_3.score(X_train,y_train)))
print('Test_score: {}'.format(grid_classifier_3.score(X_test,y_test)))

Best parameters: {'cf__fe__digit_count': True, 'cf__fe__word_count': True, 'cf__vec__preprocessor__lammetize': True, 'cf__vec__vectorizer__max_df': 0.8, 'cf__vec__vectorizer__max_features': 1000, 'classifier__scale_pos_weight': 7}
Train_score: 0.9919354838709677
Test_score: 0.9452054794520548


I'm choosing Classifier_3 as it is giving me the best f1 scores: Train score = 0.99 and Test score= 95.
Now we are going to train the pipeline using the complete dataset (spam_df). 

In the next notebook, I'm using the third pipeline with the best parameters obtained in the grid search to train the model on the full data set.